In [ ]:
# импорт зависимостей
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# настройка matplotlib
%config InlineBackend.figure_format = "retina"
plt.style.use("seaborn-v0_8-notebook")

# путь к данным
DATA_DIR = Path("../data")

##### **Чтение и построение изображений**

In [ ]:
# по умолчанию OpenCV использует BGR порядок каналов 
# чтение BGR изображения 
bgr_image = cv2.imread(
    DATA_DIR / "dog_backpack.png"
)

# чтение RGB изображения 
rgb_image = cv2.imread(
    DATA_DIR / "dog_backpack.png", 
    cv2.IMREAD_COLOR_RGB
)

# построение RGB изображения в Matplotlib
plt.figure(figsize=(5, 5))

plt.imshow(rgb_image)

# plt.axis("off")
plt.title("RBG image")
plt.xlabel("x")
plt.ylabel("y");

##### **Свойства RGB изображения**

In [ ]:
# read BGR image
image = cv2.imread(DATA_DIR / "dog_backpack.png")

print(f"type: {type(image)}")
print(f"dtype: {image.dtype}")

# высота, ширина и число каналов изображения
height, width, channels = image.shape
print(f"height x width x channels: {height} x {width} x {channels}")

##### **Чтение данных из изображения**

In [ ]:
# read RGB image
image = cv2.imread(
    DATA_DIR / "dog_backpack.png",
    cv2.IMREAD_COLOR_RGB
)

# значение пикселя с координатами i = 20, j = 50
pixel = image[20, 50]
print(f"pixel: {pixel}")

# вырежем прямоугольник (bounding-box) на изображении
# c центром 
cx = 100
cy = 200 

# размера    
w = 50
h = 70

# координаты верхнего-левого угла
x = cx - w // 2
y = cy - h // 2 

# bounding-box
bbox = image[y: y + h, x: x + w] 
print(f"bbox shape: {bbox.shape}")

# visualization
plt.figure(figsize=(4, 4))

plt.imshow(bbox)
plt.axis("off");

##### **Запись данных в изображение**

In [ ]:
# создадим серое RGB изображение
image = 127 * np.ones((512, 1024, 3), dtype=np.uint8)

# прямоугольник из красных пикселей
image[10:256, 10:512] = 255, 0, 0

# прямоугольник из зеленых пикселей
image[128:384, 256:768] = 0, 255, 0

# прямоугольник из синих пикселей
image[256:-10, 512:-10] = 0, 0, 255

# visualization
plt.figure(figsize=(6, 6))
plt.axis("off")
plt.imshow(image);

##### **Работа с цветовыми каналами**

In [ ]:
# read RGB image
image = cv2.imread(
    DATA_DIR / "dog_backpack.png",
    cv2.IMREAD_COLOR_RGB
)

# разделим изображение на цветовые каналы с помощью Numpy
R = image[:, :, 0]
G = image[:, :, 1]
B = image[:, :, 2]

# объединение каналов в Numpy
image = np.dstack((R, G, B))

# разделение на каналы в OpenCV
R, G, B = cv2.split(image)

# объединение каналов в OpenCV
image = cv2.merge((R, G, B))

# visualization
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(6, 18))

ax1.imshow(R, cmap="gray")
ax1.axis("off")
ax1.set_title("Red channel")

ax2.imshow(G, cmap="gray")
ax2.axis("off")
ax2.set_title("Green channel")

ax3.imshow(B, cmap="gray")
ax3.axis("off")
ax3.set_title("Blue channel")

plt.tight_layout()

##### **Grayscale image**

In [ ]:
# способы получить grayscale изображение

# 1) чтение из файла 
grayscale_image = cv2.imread(DATA_DIR / "lena.png", cv2.IMREAD_GRAYSCALE)

# 2) OpenCV конвертация из RGB в Grayscale
rgb_image = cv2.imread(DATA_DIR / "lena.png", cv2.IMREAD_COLOR_RGB)
grayscale_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2GRAY)

# 3) OpenCV конвертация из RGB в YCrCb 
ycrcb_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2YCrCb)
grayscale_image = ycrcb_image[:, :, 0]

# 4) вычислением Y = 0.299 * R + 0.587 * G + 0.114 * B
grayscale_image = np.uint8(rgb_image @ np.array([0.299, 0.587, 0.114]))

# построение изображений
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 16))

ax1.imshow(rgb_image)
ax1.axis("off")

ax2.imshow(grayscale_image, cmap="gray")
ax2.axis("off")

plt.tight_layout()

##### **Изменение насыщенности изображения**

$$
\begin{align*}
      \mathbf{R} &= \beta \cdot \mathbf{R} + (1-\beta)\cdot\mathbf{Y}\\
      \mathbf{G} &= \beta \cdot \mathbf{G} + (1-\beta)\cdot\mathbf{Y}\\
      \mathbf{B} &= \beta \cdot \mathbf{B} + (1-\beta)\cdot\mathbf{Y}
    \end{align*}
$$

In [ ]:
def adjust_saturation(src_img, beta):
    assert 0 <= beta <= 1

    # цветовые каналы
    R, G, B = cv2.split(src_img)

    # grayscale изображение
    Y = cv2.cvtColor(src_img, cv2.COLOR_RGB2GRAY)

    # изменение насыщенности
    R = beta * R + (1 - beta) * Y
    G = beta * G + (1 - beta) * Y
    B = beta * B + (1 - beta) * Y

    # объединение каналов в аугментированное изображение
    dst_img = cv2.merge((R, G, B))

    # преобразование из double в uint8
    dst_img = np.uint8(np.clip(dst_img, 0, 255))

    return dst_img


# исходное RGB изображение
src_img = cv2.imread(DATA_DIR / "lena.png", cv2.IMREAD_COLOR_RGB)

# Изменение насыщенности
beta = 0.4
dst_img = adjust_saturation(src_img, beta)

# visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 12))

ax1.imshow(src_img)
ax1.set_title("src_img")
ax1.axis("off")

ax2.imshow(dst_img)
ax2.set_title(f"dst_img, beta = {beta}")
ax2.axis("off")

plt.tight_layout()

##### **Цветовое пространство HSV**

In [ ]:
# читаем RGB изображение
rgb_img = cv2.imread(
    DATA_DIR / "coffee.png", 
    cv2.IMREAD_COLOR_RGB
)

# два способа конвертации в HSV
# https://docs.opencv.org/4.x/df/d9d/tutorial_py_colorspaces.html

# 1) способ 
# 0 <= Hue <= 179
# 0 <= Saturation <= 255
# 0 <= Value <= 255
hsv_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2HSV)

# 2) способ 
# 0 <= Hue <= 255
# 0 <= Saturation <= 255
# 0 <= Value <= 255
hsv_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2HSV_FULL)

# разделение на каналы
hue, saturation, value = cv2.split(hsv_img)

# visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(
    nrows=2, ncols=2, 
    figsize=(8, 5)
)

ax1.imshow(rgb_img)
ax1.set_title("RGB image")
ax1.axis("off")

ax2.imshow(hue, cmap="hsv")
ax2.set_title("Hue channel")
ax2.axis("off")

ax3.imshow(saturation, cmap="gray")
ax3.set_title("Saturation channel")
ax3.axis("off")

ax4.imshow(value, cmap="gray")
ax4.set_title("Value channel")
ax4.axis("off")

fig.tight_layout()

fig.savefig("HSV_sample.png")

##### **Вертикальные и горизонтальные отражения изображения**

In [ ]:
# read RGB image
image = cv2.imread(
    DATA_DIR / "strawberries_coffee.png", 
    cv2.IMREAD_COLOR_RGB
)

# vertical flip
vertical_flipped_image = np.flipud(image)
# vertical_flipped_image = cv2.flip(image, 0)

# horizontal flip
horizontal_flipped_image = np.fliplr(image)
# horizontal_flipped_image = cv2.flip(image, 1)

# visualization
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10, 18))

ax1.imshow(image)
ax1.set_title("original image")
ax1.axis("off")

ax2.imshow(vertical_flipped_image)
ax2.set_title("vertical flipped image")
ax2.axis("off")

ax3.imshow(horizontal_flipped_image)
ax3.set_title("horizontal flipped image")
ax3.axis("off")

plt.tight_layout()

##### **Изменения размеров изображения в OpenCV**

In [ ]:
# исходное изображение
src_img = cv2.imread(DATA_DIR / "apple.png", cv2.IMREAD_COLOR_RGB)

# коэффициенты масштабирования
fx = 2
fy = 0.5

# 1) способ 
dst_img = cv2.resize(
    src_img, 
    None, 
    # задаем коэффициенты масштабирования
    fx=fx, fy=fy, 
    # метод интерполяции 
    # cv2.INTER_NEAREST, cv2.INTER_LINEAR, cv2.INTER_CUBIC
    interpolation=cv2.INTER_NEAREST 
)

# 2) способ 
dst_img = cv2.resize(
    src_img, 
    (384, 102), # задаем размеры выходного изображения
    interpolation=cv2.INTER_LINEAR
)

# visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 12))

ax1.imshow(src_img, cmap="gray")
ax1.set_title(f"src_img {src_img.shape[:2]}")
ax1.axis("off")

ax2.imshow(dst_img,  cmap="gray")
ax2.set_title(f"dst_img {dst_img.shape[:2]}")
ax2.axis("off")

plt.tight_layout()

##### **Сдвиг изображения**

In [ ]:
# исходное изображение
src_img = cv2.imread(DATA_DIR / "lena.png", cv2.IMREAD_COLOR_RGB)
src_h, src_w = src_img.shape[:2]

# координаты вектора сдвига
tx = 100
ty = 200

# матрица преобразования сдвига
T = np.float32([
    [1, 0, tx],
    [0, 1, ty],
    [0, 0, 1]
])

# сдвиг изображения
dst_img = cv2.warpAffine(
    src_img, 
    T[:2],           # первые две строки матрицы T
    (src_w, src_h),  # размеры выходного изображения
    cv2.INTER_LINEAR # способ интерполяции
)

# visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 12))

ax1.imshow(src_img, cmap="gray")
ax1.set_title(f"src_img {src_img.shape[:2]}")
ax1.axis("off")

ax2.imshow(dst_img,  cmap="gray")
ax2.set_title(f"dst_img {dst_img.shape[:2]}")
ax2.axis("off")

plt.tight_layout()

##### **Поворот изображения относительно начала координат**

In [ ]:
# исходное изображение
src_img = cv2.imread(DATA_DIR / "lena.png", cv2.IMREAD_COLOR_RGB)
src_h, src_w = src_img.shape[:2]

# угол поворота
theta = 15
theta = np.deg2rad(theta)

# матрица поворота
R = np.float32([
    [np.cos(theta), -np.sin(theta), 0],
	[np.sin(theta),  np.cos(theta), 0],
    [0, 0, 1]
])

# поворот изображения
dst_img = cv2.warpAffine(
    src_img, 
    R[:2],           # первые две строки матрицы R
    (src_w, src_h),  # размеры выходного изображения
    cv2.INTER_LINEAR # способ интерполяции
)

# visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 12))

ax1.imshow(src_img, cmap="gray")
ax1.set_title(f"src_img {src_img.shape[:2]}")
ax1.axis("off")

ax2.imshow(dst_img,  cmap="gray")
ax2.set_title(f"dst_img {dst_img.shape[:2]}")
ax2.axis("off")

plt.tight_layout()

##### **Преобразование подобия изображения**

##### *1 способ:  явное вычисление матрицы преобразования*

In [ ]:
# исходное изображение
src_img = cv2.imread(DATA_DIR / "lena.png", cv2.IMREAD_COLOR_RGB)
src_h, src_w = src_img.shape[:2]

# центр поворота = центр изображения
cx = src_w / 2
cy = src_h / 2

# угол поворота
theta = 45
theta = np.deg2rad(theta)

# коэффициент масштаба
f = 0.6

# преобразование сдвига начала координат в центр поворота
T = np.array([
    [1, 0, -cx],
    [0, 1, -cy],
    [0, 0, 1]
])

# поворот вокруг центра поворота
R = np.array([
    [np.cos(theta), -np.sin(theta), 0],
    [np.sin(theta), np.cos(theta), 0],
    [0, 0, 1]
])

# масштабирование
F = np.array([
    [f, 0, 0],
    [0, f, 0],
    [0, 0, 1]
])

# матрица преобразования подобия
S = np.linalg.inv(T) @ F @ R @ T

# преобразования подобия изображения
dst_img = cv2.warpAffine(
    src_img, 
    S[:2],         # первые две строки матрицы S
    (src_w, src_h),  # размеры выходного изображения
    cv2.INTER_LINEAR # способ интерполяции
)

# visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 12))

ax1.imshow(src_img, cmap="gray")
ax1.set_title(f"src_img {src_img.shape[:2]}")
ax1.axis("off")

ax2.imshow(dst_img,  cmap="gray")
ax2.set_title(f"dst_img {dst_img.shape[:2]}")
ax2.axis("off")

plt.tight_layout()

##### *2 способ: вычисление матрицы преобразования с помощью OpenCV*

In [ ]:
# исходное изображение
src_img = cv2.imread(DATA_DIR / "lena.png", cv2.IMREAD_COLOR_RGB)
src_h, src_w = src_img.shape[:2]

# центр поворота = центр изображения
cx = src_w / 2
cy = src_h / 2

# угол поворота
theta = 45

# коэффициент масштаба
s = 0.6

# вычисление первых двух строк матрицы Sim в OpenCV 
M = cv2.getRotationMatrix2D((cx, cy), -theta, scale=s)

# преобразования подобия изображения
dst_img = cv2.warpAffine(
    src_img, 
    M,            
    (src_w, src_h),  # размеры выходного изображения
    cv2.INTER_LINEAR # способ интерполяции
)

# visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 12))

ax1.imshow(src_img, cmap="gray")
ax1.set_title(f"src_img {src_img.shape[:2]}")
ax1.axis("off")

ax2.imshow(dst_img,  cmap="gray")
ax2.set_title(f"dst_img {dst_img.shape[:2]}")
ax2.axis("off")

plt.tight_layout()

##### **Аффинное преобразование изображения**

In [ ]:
src_img = cv2.imread(DATA_DIR / "dog_backpack.png", cv2.IMREAD_COLOR_RGB)

# точки на исходном изображении
src_pts = np.float32([
    [260, 260],
    [730, 460],
    [610, 795]
]) / 2

# выходное изображение
h = 100
w = 150

dst_img = np.zeros((h, w, 3), dtype=np.uint8)

# точки на выходном изображении
dst_pts = np.float32([
    [5, 5],
    [w - 5, 5],
    [w - 5, h - 5]
])

# visualization
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4))

ax1.imshow(src_img)
ax1.plot(src_pts[:, 0], src_pts[:, 1], "o", color="red")

ax2.imshow(dst_img)
ax2.plot(dst_pts[:, 0], dst_pts[:, 1], "o", color="red")

plt.tight_layout()

In [ ]:
# вычислим матрицу аффинного преобразования
A = cv2.getAffineTransform(src_pts, dst_pts) 

print(f"A shape: {A.shape}")

# аффинное преобразование изображения
dst_img = cv2.warpAffine(src_img, A, (w, h), cv2.INTER_LINEAR)

# visualization
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4))

ax1.imshow(src_img)
ax1.plot(src_pts[:, 0], src_pts[:, 1], "o", color="red")

ax2.imshow(dst_img)
ax2.plot(dst_pts[:, 0], dst_pts[:, 1], "o", color="red")

plt.tight_layout()

##### **Проективное преобразование изображения**

In [ ]:
src_img = cv2.imread(DATA_DIR / "rails.jpg", cv2.IMREAD_COLOR_RGB)

# точки на исходном изображении
src_pts = np.float32([
    [60, 780],
    [680, 420],
    [975, 420],
    [1590, 780]
])

# выходное изображение
h = 360
w = 680

dst_img = np.zeros((h, w, 3), dtype=np.uint8)

# точки на выходном изображении
dst_pts = np.float32([
    [0, 0],
    [w, 0],
    [w, h],
    [0, h]
])

# visualization
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4))

ax1.imshow(src_img)
ax1.plot(src_pts[:, 0], src_pts[:, 1], "o", color="red")

ax2.imshow(dst_img)
ax2.plot(dst_pts[:, 0], dst_pts[:, 1], "o", color="red")

plt.tight_layout()

In [ ]:
# вычислим матрицу гомографии
H = cv2.getPerspectiveTransform(src_pts, dst_pts) 
print(f"H shape: {H.shape}")

# проективное преобразование изображения
dst_img = cv2.warpPerspective(src_img, H, (w, h), cv2.INTER_LINEAR)

# visualization
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4))

ax1.imshow(src_img)
ax1.plot(src_pts[:, 0], src_pts[:, 1], "o", color="red")

ax2.imshow(dst_img)
ax2.plot(dst_pts[:, 0], dst_pts[:, 1], "o", color="red")

plt.tight_layout()

##### **Преобразование координат при проективном отображении**

$$
    \begin{bmatrix*}[c]
        x'_1 \\ x'_2 \\ x'_3    
    \end{bmatrix*}=
    \mathbf{H}
    \begin{bmatrix*}[c]
        x \\ y \\ 1    
    \end{bmatrix*}
$$
$$
    \begin{bmatrix*}[c]
        x_1 \\ x_2 \\ x_3    
    \end{bmatrix*}=
    \mathbf{H}^{-1}
    \begin{bmatrix*}[c]
        x' \\ y' \\ 1    
    \end{bmatrix*}
$$

In [ ]:
# евклидовы координаты точки на dst_img
x_ = 300
y_ = 200

# однородные координаты точки на dst_img
dst_point = x_, y_, 1

# обратное проективное отображение
Hinv = np.linalg.inv(H)

# преобразование однородных координат точки
src_point = Hinv @ dst_point

# преобразование однородных координат в евклидовы
x = src_point[0] / src_point[2]
y = src_point[1] / src_point[2]

# visualization
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4))

ax1.imshow(src_img)
ax1.plot(x, y, "o", color="red")
ax1.set_title("src_img")

ax2.imshow(dst_img)
ax2.plot(x_, y_, "o", color="red")
ax2.set_title("dst_img")

plt.tight_layout()

#### **Домашняя работа 1**

##### **Задача**

Используя OpenCV, напишите программу ``homeworks/homework_01.py``, позволяющую:

- выделять четыре точки на видео с камеры, которые задают четырехугольную область

- с помощью билинейной гомографии преобразовывать четырехугольную область в прямоугольную область

- выполнять ColorJitter аугментацию прямоугольной области 

##### **Теория**

- RGB изображение
- цветовое пространство YCrCb
- цветовое пространство HSV
- ColorJitter аугментация
- евклидовы и однородные координаты на плоскости
- преобразование подобия
- аффинное преобразование
- проективное преобразование (гомография)